In [18]:
import random
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import GroupKFold
import pandas as pd
import math
import time
import os


In [ ]:
from rdflib import Graph, URIRef, Literal, RDF, ConjunctiveGraph, Namespace

# Get the features and gold standard 

In [19]:
drugfeatfiles = ['drugs-fingerprint-sim.csv','drugs-se-sim.csv', 
                     'drugs-ppi-sim.csv', 'drugs-target-go-sim.csv','drugs-target-seq-sim.csv']
diseasefeatfiles =['diseases-hpo-sim.csv',  'diseases-pheno-sim.csv' ]



In [20]:
feature_folder="data/features/"

In [21]:
drugfeatfiles = [ os.path.join(feature_folder, fn) for fn in drugfeatfiles]
diseasefeatfiles = [ os.path.join(feature_folder, fn) for fn in diseasefeatfiles]

In [23]:
#goldindfile = 'data/input/openpredict-omim-drug.csv'
drug_ind="data/input/openpredict-omim-drug.csv"
drugDiseaseKnown = pd.read_csv(drug_ind,delimiter=',') 
drugDiseaseKnown.head()

,drugid,omimid
0,DB01148,231200
1,DB01148,155100
2,DB01148,273800
3,DB00575,607554
4,DB00575,171300


In [24]:
drugDiseaseKnown.rename(columns={'drugid':'Drug','omimid':'Disease'}, inplace=True)
drugDiseaseKnown.Disease = drugDiseaseKnown.Disease.astype(str)
drugDiseaseKnown.head()

,Drug,Disease
0,DB01148,231200
1,DB01148,155100
2,DB01148,273800
3,DB00575,607554
4,DB00575,171300


# Merge feature matrix

In [25]:
def adjcencydict2matrix(df, name1, name2):
    df1 = df.copy()
    df1= df1.rename(index=str, columns={name1: name2, name2: name1})
    print (len(df))
    df =df.append(df1)
    print (len(df))
    return df.pivot(index=name1, columns=name2)

def mergeFeatureMatrix(drugfeatfiles, diseasefeatfiles):
    for i,featureFilename in enumerate(drugfeatfiles):
        print (featureFilename)
        df = pd.read_csv(featureFilename, delimiter=',')
        cond = df.Drug1 > df.Drug2
        df.loc[cond, ['Drug1', 'Drug2']] = df.loc[cond, ['Drug2', 'Drug1']].values
        if i != 0:
            drug_df=drug_df.merge(df,on=['Drug1','Drug2'],how='inner')
            #drug_df=drug_df.merge(temp,how='outer',on='Drug')
        else:
            drug_df =df
    drug_df.fillna(0, inplace=True)
    
    drug_df = adjcencydict2matrix(drug_df, 'Drug1', 'Drug2')
    drug_df = drug_df.fillna(1.0)

    
    for i,featureFilename in enumerate(diseasefeatfiles):
        print (featureFilename)
        df=pd.read_csv(featureFilename, delimiter=',')
        cond = df.Disease1 > df.Disease2
        df.loc[cond, ['Disease1','Disease2']] = df.loc[cond, ['Disease2','Disease1']].values
        if i != 0:
            disease_df = disease_df.merge(df,on=['Disease1','Disease2'], how='inner')
            #drug_df=drug_df.merge(temp,how='outer',on='Drug')
        else:
            disease_df = df
    disease_df.fillna(0, inplace=True)
    disease_df.Disease1 = disease_df.Disease1.astype(str)
    disease_df.Disease2 = disease_df.Disease2.astype(str)
    
    disease_df = adjcencydict2matrix(disease_df, 'Disease1', 'Disease2')
    disease_df = disease_df.fillna(1.0)
    
    return drug_df, disease_df

In [26]:
drug_df, disease_df = mergeFeatureMatrix(drugfeatfiles, diseasefeatfiles)

data/features/drugs-fingerprint-sim.csv
data/features/drugs-se-sim.csv
data/features/drugs-ppi-sim.csv
data/features/drugs-target-go-sim.csv
data/features/drugs-target-seq-sim.csv
129795
259590


/var/folders/n0/wfv9tgj96fg1rw1kvy3tmls80000gn/T/ipykernel_7458/3061590088.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df =df.append(df1)


data/features/diseases-hpo-sim.csv
data/features/diseases-pheno-sim.csv
44850
89700


/var/folders/n0/wfv9tgj96fg1rw1kvy3tmls80000gn/T/ipykernel_7458/3061590088.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df =df.append(df1)


In [27]:
drug_df.head()

TC                                                              \
Drug2     DB00014   DB00035   DB00091   DB00104   DB00122   DB00125   DB00136   
Drug1                                                                           
DB00014  1.000000  0.732558  0.574713  0.655172  0.313953  0.361446  0.402299   
DB00035  0.732558  1.000000  0.544304  0.697368  0.240506  0.441176  0.289157   
DB00091  0.574713  0.544304  1.000000  0.539474  0.365079  0.303030  0.397059   
DB00104  0.655172  0.697368  0.539474  1.000000  0.256757  0.347826  0.378378   
DB00122  0.313953  0.240506  0.365079  0.256757  1.000000  0.291667  0.267857   

                                       ... TARGETSEQ-SIM                      \
Drug2     DB00153   DB00162   DB00165  ...       DB01576   DB01577   DB01579   
Drug1                                  ...                                     
DB00014  0.321839  0.244186  0.376471  ...      0.076672  0.148647  0.039052   
DB00035  0.219512  0.150000  0.378378  ...      0.024000  0.030028  0.023312   
DB00091  0.294118  0.250000  0.384615  ...      0.039643  0.045992  0.036710   
DB00104  0.283784  0.208333  0.469697  ...      0.178035  0.180648  0.048190   
DB00122  0.280000  0.400000  0.200000  ...      0.040534  0.040534  0.041460   

                                                                               
Drug2     DB01590   DB01623   DB02546   DB04272   DB04844   DB04861   DB06287  
Drug1                                                                          
DB00014  0.028215  0.081568  0.043257  0.053699  0.053920  0.113543  0.028215  
DB00035  0.043115  0.023471  0.033298  0.031706  0.024000  0.021341  0.043115  
DB00091  0.016369  0.031936  0.059601  0.073739  0.039643  0.047390  0.016369  
DB00104  0.015979  0.172655  0.040032  0.052726  0.054976  0.188184  0.019203  
DB00122  0.027075  0.033040  0.050565  0.065972  0.036566  0.042961  0.035405  

[5 rows x 3060 columns]

In [28]:
disease_df.head()

HPO-SIM                                                    \
Disease2    102100    102300    102400    102500    103100    103780   
Disease1                                                               
102100    1.000000  0.228439  0.420190  0.432394  0.365865  0.091080   
102300    0.228439  1.000000  0.291239  0.257371  0.480403  0.427043   
102400    0.420190  0.291239  1.000000  0.464588  0.334854  0.083732   
102500    0.432394  0.257371  0.464588  1.000000  0.266952  0.221382   
103100    0.365865  0.480403  0.334854  0.266952  1.000000  0.322904   

                                                  ... PHENO-SIM            \
Disease2    104130    104300    106400    107320  ...    607682    607685   
Disease1                                          ...                       
102100    0.312120  0.210254  0.516519  0.341580  ...  0.050063  0.067166   
102300    0.425632  0.510889  0.326918  0.204134  ...  0.084685  0.121191   
102400    0.257330  0.273490  0.484795  0.210790  ...  0.060523  0.054133   
102500    0.427080  0.263057  0.439328  0.184396  ...  0.123718  0.092214   
103100    0.426191  0.459806  0.370533  0.453242  ...  0.109109  0.081325   

                                                                      \
Disease2    607850    608033    608088    608105    608217    608320   
Disease1                                                               
102100    0.132453  0.102598  0.216295  0.108148  0.132453  0.135302   
102300    0.179244  0.277684  0.182940  0.146352  0.089622  0.183099   
102400    0.106752  0.082690  0.130744  0.130744  0.160128  0.081786   
102500    0.072739  0.197203  0.089087  0.111359  0.163663  0.153252   
103100    0.128300  0.099381  0.157135  0.117851  0.096225  0.172016   

                              
Disease2    608583    608622  
Disease1                      
102100    0.155113  0.183942  
102300    0.236148  0.124461  
102400    0.093761  0.148250  
102500    0.127775  0.101015  
103100    0.140859  0.222718  

[5 rows x 900 columns]

# Generate positive and negative pairs

In [30]:
def generatePairs(drug_df, disease_df, drugDiseaseKnown):
    drugwithfeatures = set(drug_df.columns.levels[1])
    diseaseswithfeatures = set(disease_df.columns.levels[1])
    
    drugDiseaseDict  = set([tuple(x) for x in  drugDiseaseKnown[['Drug','Disease']].values])

    commonDrugs= drugwithfeatures.intersection( drugDiseaseKnown.Drug.unique())
    commonDiseases=  diseaseswithfeatures.intersection(drugDiseaseKnown.Disease.unique() )
    print ("commonDrugs: %d commonDiseases : %d"%(len(commonDrugs),len(commonDiseases)))

    #abridged_drug_disease = [(dr,di)  for  (dr,di)  in drugDiseaseDict if dr in drugwithfeatures and di in diseaseswithfeatures ]

    #commonDrugs = set( [ dr  for dr,di in  abridged_drug_disease])
    #commonDiseases  =set([ di  for dr,di in  abridged_drug_disease])

    print ("Gold standard, associations: %d drugs: %d diseases: %d"%(len(drugDiseaseKnown),len(drugDiseaseKnown.Drug.unique()),len(drugDiseaseKnown.Disease.unique())))
    print ("Drugs with features: %d Diseases with features: %d"%(len(drugwithfeatures),len(diseaseswithfeatures)))
    print ("commonDrugs: %d commonDiseases : %d"%(len(commonDrugs),len(commonDiseases)))

    pairs=[]
    classes=[]
    for dr in commonDrugs:
        for di in commonDiseases:
            cls = (1 if (dr,di) in drugDiseaseDict else 0)
            pairs.append((dr,di))
            classes.append(cls)
            
    return pairs, classes

In [31]:
pairs, classes = generatePairs(drug_df, disease_df, drugDiseaseKnown)

commonDrugs: 510 commonDiseases : 300
Gold standard, associations: 1933 drugs: 592 diseases: 313
Drugs with features: 510 Diseases with features: 300
commonDrugs: 510 commonDiseases : 300


# Balance negative samples/postives 

In [32]:
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold
def balance_data(pairs, classes, n_proportion):
    classes = np.array(classes)
    pairs = np.array(pairs)
    
    indices_true = np.where(classes == 1)[0]
    indices_false = np.where(classes == 0)[0]

    np.random.shuffle(indices_false)
    indices = indices_false[:(n_proportion*indices_true.shape[0])]
    print ("+/-:", len(indices_true), len(indices), len(indices_false))
    pairs = np.concatenate((pairs[indices_true], pairs[indices]), axis=0)
    classes = np.concatenate((classes[indices_true], classes[indices]), axis=0) 
    
 
    return pairs, classes

In [33]:
n_proportion = 2
pairs, classes= balance_data(pairs, classes, n_proportion)

+/-: 1671 3342 151329


# Train-Test Splitting

In [34]:
pairs_train, pairs_test, classes_train, classes_test = model_selection.train_test_split(pairs, classes, stratify=classes, test_size=0.2, shuffle=True)

In [36]:
len(pairs_train), len(pairs_test)

(4010, 1003)

# Feature extraction (Best Single Feature Similarity)

In [38]:

def calculateDrugMaxMean(drug, disease, knownDrugDisease, drugDF):
    #print (drug, disease)
    
    # get only diseases related to this drug
    filteredDrugs=knownDrugDisease[knownDrugDisease[:,1]==disease,0]
    similarities  = drugDF.loc[filteredDrugs][drug].values
    similarities2= np.where(similarities==1.0,0.0,similarities)
    #knownDrugDisease[knownDrugDisease[:,1]==disease,0]
    #c=np.where(a==1.0,0.0,a)
    try:
        maxSimilarity=float(np.max(similarities2))
    except :
        maxSimilarity=0.0         
        
    return maxSimilarity


def calculateDiseaseMaxMean(drug, disease, knownDrugDisease, diseaseDF):
    #print (drug, disease)
    b  = diseaseDF.loc[knownDrugDisease[:,1]][disease].values
    #b= np.sqrt( np.multiply(b,b) ) #remove negative values
    
    c=np.where(b==1.0,0.0,b)
    return float(np.max(c))
 


def createSingleFeatureDF(pairs, classes, knownDrugDisease, drugDFs, diseaseDFs):
    totalNumFeatures = len(drugDFs)*len(diseaseDFs)
    #featureMatri x= np.empty((len(classes),totalNumFeatures), float)
    df =pd.DataFrame(list(zip(pairs[:,0], pairs[:,1], classes)), columns =['Drug','Disease','Class'])
    index = 0
    for i,drug_col in enumerate(drugDFs.columns.levels[0]):
        drugDF = drugDFs[drug_col]
        df["Feature_"+str(drug_col)] = df.apply(lambda row: calculateDrugMaxMean( row.Drug, row.Disease, knownDrugDisease, drugDF), axis=1)
        
    for j,disease_col in enumerate(diseaseDFs.columns.levels[0]):
        diseaseDF = diseaseDFs[disease_col]
        df["Feature_"+str(disease_col)] = df.apply(lambda row: calculateDiseaseMaxMean( row.Drug, row.Disease, knownDrugDisease, diseaseDF), axis=1)
    return df


def calculateSingleSimilarity(pairs_train, pairs_test, classes_train, classes_test, drug_df, disease_df, knownDrugDisease):
    train_df  = createSingleFeatureDF(pairs_train, classes_train, knownDrugDisease, drug_df, disease_df)
    test_df = createSingleFeatureDF(pairs_test, classes_test, knownDrugDisease, drug_df, disease_df)
    return train_df, test_df


In [39]:
knownDrugDisease= pairs_train[classes_train==1]

train_df, test_df = calculateSingleSimilarity(pairs_train, pairs_test, classes_train, classes_test, drug_df, disease_df, knownDrugDisease)


# Model Training

In [40]:
from sklearn import tree, ensemble
from sklearn import svm, linear_model, neighbors

def trainModel(train_df, clf):
    #features = list(train_df.columns.difference(['Drug','Disease','Class']))
    features= ['Feature_GO-SIM',
        'Feature_PPI-SIM',
        'Feature_SE-SIM',
        'Feature_TARGETSEQ-SIM',
        'Feature_TC',
        'Feature_HPO-SIM',
        'Feature_PHENO-SIM']
    X = train_df[features]
    y = train_df['Class']
    print ('fiting classifier...')
    clf.fit(X, y)
    return clf

In [52]:
n_seed = 100
clf = linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, random_state=n_seed) 
clf = trainModel(train_df, clf)

fiting classifier...


# Evaulation 

In [54]:
from sklearn import metrics
import numbers
def multimetric_score(estimator, X_test, y_test, scorers):
    """Return a dict of score for multimetric scoring"""
    scores = {}
    for name, scorer in scorers.items():
        if y_test is None:
            score = scorer(estimator, X_test)
        else:
            
            score = scorer(estimator, X_test, y_test)

        if hasattr(score, 'item'):
            try:
                # e.g. unwrap memmapped scalars
                score = score.item()
            except ValueError:
                # non-scalar?
                pass
        scores[name] = score

        if not isinstance(score, numbers.Number):
            raise ValueError("scoring must return a number, got %s (%s) "
                             "instead. (scorer=%s)"
                             % (str(score), type(score), name))
    return scores

def evaluate(test_df, clf):
    #
    # features = list(train_df.columns.difference(['Drug','Disease','Class']))
    features= ['Feature_GO-SIM',
        'Feature_PPI-SIM',
        'Feature_SE-SIM',
        'Feature_TARGETSEQ-SIM',
        'Feature_TC',
        'Feature_HPO-SIM',
        'Feature_PHENO-SIM']
    X_test =  test_df[features]
    y_test = test_df['Class']

    scoring = ['precision', 'recall', 'accuracy', 'roc_auc', 'f1', 'average_precision']
    #scorers, multimetric = metrics.scorer._check_multimetric_scoring(clf, scoring=scoring)
    scorers = {}
    for scorer in scoring:
        scorers[scorer] = metrics.get_scorer(scorer)

    scores = multimetric_score(clf, X_test, y_test, scorers)
    return scores

disjoint = True
n_fold = 10

In [59]:
scores = evaluate(test_df, clf)
print ("Test:",scores)

Test: {'precision': 0.7846153846153846, 'recall': 0.6107784431137725, 'accuracy': 0.8143712574850299, 'roc_auc': 0.8221341747642439, 'f1': 0.6868686868686869, 'average_precision': 0.7984775839698814}


# 10-fold drug-disjoint cross-validation (PREDICT - CV scheme )

In [57]:
disjoint = True
n_fold = 10

if disjoint:
    print ('Disjoint')
    groups = pairs[:,0] # group by drug
    group_kfold = GroupKFold(n_splits=n_fold)
    cv = group_kfold.split(pairs, classes, groups)
else:
    print ('Non-disjoint')
    skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=n_seed)
    cv = skf.split(pairs, classes)

n_seed = 100
cv_results = pd.DataFrame()
clf = linear_model.LogisticRegression(penalty='l2', solver='lbfgs', dual=False, tol=0.0001, C=1.0, random_state=n_seed) 
  
for i, (train, test) in enumerate(cv):
    print ('Fold',i+1)
    start_time = time.time()
    pairs_train = pairs[train]
    classes_train = classes[train] 
    pairs_test = pairs[test]
    classes_test = classes[test]
    knownDrugDisease= pairs_train[classes_train==1]
    
    train_df, test_df = calculateSingleSimilarity(pairs_train, pairs_test, classes_train, classes_test, drug_df, disease_df, knownDrugDisease)
    elapsed_time = time.time() - start_time
    print ('Time elapsed to generate features:',time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

    clf = trainModel(train_df, clf)
    
    scores = evaluate(test_df, clf)
    #print ("Scores:",scores)
    cv_results = cv_results.append(scores, ignore_index=True)

Disjoint
Fold 1
Time elapsed to generate features: 00:00:32
fiting classifier...
Fold 2


/var/folders/n0/wfv9tgj96fg1rw1kvy3tmls80000gn/T/ipykernel_7458/1104912358.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cv_results = cv_results.append(scores, ignore_index=True)


Time elapsed to generate features: 00:00:32
fiting classifier...
Fold 3


/var/folders/n0/wfv9tgj96fg1rw1kvy3tmls80000gn/T/ipykernel_7458/1104912358.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cv_results = cv_results.append(scores, ignore_index=True)


Time elapsed to generate features: 00:00:31
fiting classifier...
Fold 4


...

/var/folders/n0/wfv9tgj96fg1rw1kvy3tmls80000gn/T/ipykernel_7458/1104912358.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cv_results = cv_results.append(scores, ignore_index=True)


Time elapsed to generate features: 00:00:31
fiting classifier...
Fold 10


/var/folders/n0/wfv9tgj96fg1rw1kvy3tmls80000gn/T/ipykernel_7458/1104912358.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cv_results = cv_results.append(scores, ignore_index=True)


Time elapsed to generate features: 00:00:31
fiting classifier...


/var/folders/n0/wfv9tgj96fg1rw1kvy3tmls80000gn/T/ipykernel_7458/1104912358.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cv_results = cv_results.append(scores, ignore_index=True)


In [58]:
cv_results.mean()

precision            0.823548
recall               0.585959
accuracy             0.818888
roc_auc              0.827370
f1                   0.681740
average_precision    0.787859
dtype: float64

In [ ]:
os.mkdir('results')
cv_results.to_csv('results/disjoint_lr.csv')

In [2]:
#import pandas as pd
#cv_results = pd.read_csv('results/disjoint_lr.csv')

In [3]:
cv_results.head()

,Unnamed: 0,accuracy,average_precision,f1,precision,recall,roc_auc
0,0,0.734531,0.814619,0.512821,0.933333,0.353535,0.848552
1,1,0.816733,0.844710,0.671429,0.940000,0.522222,0.861680
2,2,0.814741,0.851837,0.706625,0.888889,0.586387,0.886517
3,3,0.816733,0.816190,0.701299,0.857143,0.593407,0.849579
4,4,0.868526,0.846952,0.731707,0.947368,0.596026,0.874493


In [ ]:
import time
def generateURI(prefix):
    uniqueID= int(round(time.time() * 1000))
    uri = URIRef(prefix+str(uniqueID))
    return uri

In [ ]:
DC = Namespace("http://purl.org/dc/terms/")
MLS = Namespace("http://www.w3.org/ns/mls#")
RPC = Namespace("https://w3id.org/reproduceme#")
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")

g =  ConjunctiveGraph(identifier = URIRef('http://bio2rdf.org/openpredict_resource:bio2rdf.dataset.openpredict.R1')) 

#graphURI = URIRef('http://bio2rdf.org/openpredict_resource:bio2rdf.output.openpredict.R1')
runURI = generateURI('http://www.w3.org/ns/mls#Run')
evalURI = generateURI('http://www.w3.org/ns/mls#ModelEvaluation')
evalSpecURI = generateURI('http://www.w3.org/ns/mls#EvaluationSpecification')

g.add((runURI, RDF['type'], MLS['Run']))
g.add((runURI, MLS['achieves'], RPC['Pipeline_OpenPREDICT']))

g.add((runURI, MLS['hasOutput'], evalURI))
g.add((evalURI, RDF['type'], MLS['ModelEvaluation']))

g.add((evalSpecURI, MLS['defines'],  RPC['Pipeline_OpenPREDICT']))
g.add((evalSpecURI, MLS['hasPart'],  MLS['TenFoldCrossValidation']))
g.add((MLS['TenFoldCrossValidation'], RDF['type'],  MLS['EvaluationProcedure']))
g.add((MLS['TenFoldCrossValidation'], RDFS['label'],  Literal('10-fold CV')))

g.add((evalSpecURI, MLS['hasPart'],  MLS['DrugwiseCrossValidation']))
g.add((MLS['DrugwiseCrossValidation'], RDF['type'],  MLS['EvaluationProcedure']))
g.add((MLS['DrugwiseCrossValidation'], RDFS['label'],  Literal('Drugwise CrossValidation')))
g.add((MLS['DrugwiseCrossValidation'], DC['description'],  Literal('Split drugs in 10-fold, remove drugs of each fold in the gold standard and consequently remove all the known indication sassociated with them')))

for index, value in cv_results.mean().items():
    measureURI = generateURI('http://www.w3.org/ns/mls#Measure_'+index)
    g.add((evalSpecURI, MLS['hasPart'],  measureURI))      
    g.add((evalURI, MLS['specifiedBy'],measureURI ))
    g.add((measureURI, RDF['type'], MLS['EvaluationMeasure']))
    g.add((measureURI, RDFS['label'],  Literal(index)))
    g.add((measureURI, MLS['hasValue'],  Literal(value)))



In [ ]:
outfile ='results/results_disjoint_lr.nq'
g.serialize(outfile, format='nquads')
print('RDF is generated at '+outfile)

In [ ]:
#import shutil
#import os
#shutil.copytree('results',os.path.join(abs_path, 'data/results/'))